### Step 2 - Make it in directory

In [ ]:
!mkdir ./dataset/train

In [ ]:
!mkdir ./dataset/train/healthy
!mkdir ./dataset/train/late_blight
!mkdir ./dataset/train/early_blight

In [ ]:
!cp ./potato/Africa_dataset/Africa/Potato___healthy/. ./dataset/train/healthy/ -r

In [ ]:
!cp ./potato/Africa_dataset/Africa/Potato___Late_blight/. ./dataset/train/late_blight/ -r

In [ ]:
!cp ./potato/Peru_dataset/Peru/Potato___Late_blight/. ./dataset/train/late_blight/ -r
!cp ./potato/Peru_dataset/Peru/Potato___Early_blight/. ./dataset/train/early_blight/ -r
!cp ./potato/Peru_dataset/Peru/Potato___healthy/. ./dataset/train/healthy/ -r

In [ ]:
!cp ./potato/internet_dataset/potato_test_1_Corriger/Potato___Late_blight/. ./dataset/train/late_blight/ -r
!cp ./potato/internet_dataset/potato_test_1_Corriger/Potato___Early_blight/. ./dataset/train/early_blight/ -r
!cp ./potato/internet_dataset/potato_test_1_Corriger/Potato___healthy/. ./dataset/train/healthy/ -r

In [ ]:
!cp "./potato_int/Late Blight/." ./dataset/train/late_blight/ -r
!cp ./potato_int/Healthy/. ./dataset/train/healthy/ -r

## 2.2 - Pre-Process Photos into Standard Directories

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import keras

train_datagen = ImageDataGenerator ( rescale =1./255 ,
shear_range =0.2 ,
zoom_range =0.2 ,
horizontal_flip = True ,
validation_split =0.2) # set validation split

train_generator = train_datagen.flow_from_directory (
"/content/dataset/train",# dataset path in drive
target_size = (300 , 300) ,
color_mode ="grayscale",
shuffle = True ,
batch_size =32 ,
class_mode ="categorical",
subset ="training") # set as training data

validation_generator = train_datagen . flow_from_directory (
r"./dataset/train", # dataset path in drive
target_size = (300 , 300) ,
color_mode ="grayscale",
batch_size =32 ,
shuffle = True ,
class_mode ="categorical",
subset ="validation") # set as validation data

Found 853 images belonging to 3 classes.
Found 211 images belonging to 3 classes.


# 3 - Develop a Baseline CNN Model

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(300, 300, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 16)       0

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') >= 0.9): # Experiment with changing this value
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True
callbacks = myCallback()      


model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])



In [ ]:
history = model.fit(
      train_generator,

      batch_size=10,  
      epochs=25,
      verbose=1)

Epoch 1/25
27/27 [==============================] - 86s 3s/step - loss: 1.0532 - accuracy: 0.4502
Epoch 2/25
27/27 [==============================] - 72s 3s/step - loss: 1.0185 - accuracy: 0.4678
Epoch 3/25
27/27 [==============================] - 72s 3s/step - loss: 0.9631 - accuracy: 0.5275
Epoch 4/25
27/27 [==============================] - 73s 3s/step - loss: 0.8759 - accuracy: 0.5909
Epoch 5/25
27/27 [==============================] - 72s 3s/step - loss: 0.8329 - accuracy: 0.6155
Epoch 6/25
27/27 [==============================] - 72s 3s/step - loss: 0.7916 - accuracy: 0.6401
Epoch 7/25
27/27 [==============================] - 73s 3s/step - loss: 0.7506 - accuracy: 0.6811
Epoch 8/25
27/27 [==============================] - 72s 3s/step - loss: 0.7020 - accuracy: 0.6999
Epoch 9/25
27/27 [==============================] - 74s 3s/step - loss: 0.6690 - accuracy: 0.7104
Epoch 10/25
27/27 [==============================] - 73s 3s/step - loss: 0.6239 - accuracy: 0.7573
Epoch 11/25
27/27 [

In [ ]:
model.evaluate(
        validation_generator,
        verbose=1
    )

7/7 [==============================] - 21s 3s/step - loss: 0.9687 - accuracy: 0.7014


[0.968728244304657, 0.7014217972755432]

#submit model

In [ ]:

test_real_datagen = ImageDataGenerator (rescale =1./255 ,) # set validation split

test_real_generator = test_real_datagen.flow_from_directory(
  "/content/potato/Farmy_test/",# dataset path in drive
  target_size = (300 , 300),
  color_mode ="grayscale",
  shuffle = True,
  batch_size =32,
  class_mode =None,
) # set as training data

Found 224 images belonging to 1 classes.


In [ ]:
predictions= model.predict(
        test_real_generator,
        verbose=1
    )

7/7 [==============================] - 6s 815ms/step


In [ ]:
len(predictions)

224

In [ ]:
import numpy as np

predictions[1]



array([0.39574876, 0.4137109 , 0.19054033], dtype=float32)

In [ ]:
np.argmax(predictions[1])

2

In [ ]:
train_generator.class_indices

{'early_blight': 0, 'healthy': 1, 'late_blight': 2}

In [ ]:
list(train_generator.class_indices.keys())[list(train_generator.class_indices.values()).index(0)].replace('_',' ')


'early blight'

In [ ]:
import pandas as pd

df=pd.DataFrame(columns=['image','Classification target'])

for i in range(224):
  df=df.append({'image':test_real_generator.filenames[i].replace('farmy/',''),
                'Classification target':list(train_generator.class_indices.keys())[list(train_generator.class_indices.values()).index(np.argmax(predictions[i]))].replace('_',' ')
                },ignore_index=True)



In [ ]:
df

,image,Classification target
0,01126e2630b15e7e3a0a26869d47ed477e0b7f6c.jpeg,late blight
1,01a83bd0e39561fc090a2dd8f090e15866ddff7d.jpeg,healthy
2,024f92e0e4991f95835d994e7cdc68545fa643c5.jpeg,healthy
3,0571a2fd5825c2b5da83b96bec5da285cbab79b8.jpeg,healthy
4,06b15764355d627b366c4ff5dae2ecd89a9d79fc.jpeg,healthy
...,...,...
219,fb981715bba6c68931052a5c576809750786bb52.jpeg,healthy
220,fd6faa4f69ad7b5f5db5006863b356d2897ff0e3.jpeg,healthy
221,fe5e25797e6fe7f20d2cfdc4873d75ee169c1196.jpeg,late blight
222,ff2abe3fe17feb5da8fef7193b999e3078619671.jpeg,healthy


In [ ]:
import calendar
import time
ts=calendar.timegm(time.gmtime())

In [ ]:
df.to_csv(str(ts)+'_results.csv',index=False)